In [1]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

print(os.environ.get('JAVA_HOME'))


from pyserini.search.lucene import LuceneSearcher
from datasets import get_dataset
import ir_datasets




/usr/lib/jvm/java-11-openjdk-amd64


/home/catalinlup/anaconda3/envs/ranking/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

In [3]:
searcher.set_bm25(0.82, 0.68)

In [4]:
# get the queries
# psg20 = ir_datasets.load("msmarco-passage/dev/small")
# queries = psg20.queries_iter()

queries, qrels = get_dataset('trec-dl-2019-sample')

qids = []
query_text = []
for q in queries:
    qids.append(q.query_id)
    query_text.append(q.text)

# doc_ids = set(map(lambda x: x.doc_id, qrels))

In [5]:
K = 5000
hits = searcher.batch_search(queries=query_text, qids=qids, k=K, threads=6)



In [6]:
query_id = []
document_id = []
rank = []
score = []
for qid in qids:
    for i in range(len(hits[qid])):

        # if hits[qid][i].docid not in doc_ids:
        #     continue
        
        query_id.append(qid)
        document_id.append(hits[qid][i].docid)
        rank.append(i)
        score.append(hits[qid][i].score)        

In [7]:
import pandas as pd
run_file = pd.DataFrame(columns=["query_id", "run", "document_id", "rank", "score", "engine"])
run_file["query_id"] = query_id
run_file["run"] = "Q0"
run_file["document_id"] = document_id
run_file["rank"] = rank
run_file["score"] = score
run_file["engine"] = "Aserini"

print(run_file)

      query_id run document_id  rank      score   engine
0       436600  Q0     8473863     0  14.605000  Aserini
1       436600  Q0     2267986     1  14.143000  Aserini
2       436600  Q0     3813285     2  13.464800  Aserini
3       436600  Q0     2925061     3  13.023300  Aserini
4       436600  Q0     4541820     4  12.771900  Aserini
...        ...  ..         ...   ...        ...      ...
24995   443396  Q0     5308631  4995   4.731948  Aserini
24996   443396  Q0     5340110  4996   4.731947  Aserini
24997   443396  Q0     5366024  4997   4.731946  Aserini
24998   443396  Q0     5426424  4998   4.731945  Aserini
24999   443396  Q0     5538992  4999   4.731944  Aserini

[25000 rows x 6 columns]


In [8]:
run_file.to_csv("/home/catalinlup/MyWorkspace/MasterThesis/datasets/run_files/msmarco_psg_bm25_rankings.trec_2019.sample.small.tsv", sep="\t", index=False, header=False)